WordVectorz=dict(zip(model.wv.index2word,model.wv.vectors))
class AverageEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim =100 # because we use 100 embedding points 

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])
AverageEmbeddingVectorizer(WordVectorz).fit(x,y)
x.text = AverageEmbeddingVectorizer(WordVectorz).transform(x.text)

In [1]:
import pandas as pd
combined_dataset = pd.read_pickle('../combined_dataset_notes.pkl')


combined_dataset = combined_dataset.drop(['HADM_ID'], axis=1)
print(combined_dataset.shape)
combined_dataset.head()

only_text = combined_dataset[["text","G1","G2","G3","G4","G5","G6","G7","G8","G9","G10","G11","G12","G13","G14","G15","G16","G17","G18","G19","G20"]]
only_text.shape

!pip install --upgrade gensim
!pip install -U gensim


from gensim.models import Word2Vec

#pass text here..
#model = Word2Vec(combined_dataset.text,size=300, window=10, min_count=5, workers=56)#play around with param
# summarize the loaded model
#print(model)
# summarize vocabulary
#words = list(model.wv.vocab)
#print(words)
# access vector for one word
#print(model['sentence'])
# save model
#model.save('model300-10-5.bin')
# load model
new_model = Word2Vec.load('model750-10-5.bin')
print(new_model)

y = only_text.iloc[:,1:]
x = only_text.iloc[:,:1]

y.shape

'''%matplotlib inline
from sklearn.decomposition import PCA
from matplotlib import pyplot
X = model[model.wv.vocab]
pca = PCA(n_components=2)
result = pca.fit_transform(X)
# create a scatter plot of the projection
pyplot.scatter(result[:, 0], result[:, 1])
words = list(model.wv.vocab)
for i, word in enumerate(words):
    pyplot.annotate(word, xy=(result[i, 0], result[i, 1]))
pyplot.show()'''

#voila

#avg_vec.shape

x.columns

import numpy as np



#x_train,x_val,y_train,y_val = train_test_split(x_tra,y_tra,test_size=0.25,random_state=seed)
#print(x_train.shape,x_val.shape,y_train.shape,y_test.shape)

(45526, 504)
Requirement already up-to-date: gensim in /home/akshara/anaconda3/lib/python3.7/site-packages (3.8.1)
Requirement already up-to-date: gensim in /home/akshara/anaconda3/lib/python3.7/site-packages (3.8.1)
Word2Vec(vocab=23960, size=750, alpha=0.025)


In [2]:
x_train

,text
10603,"[pm, chest, pa, lat, clip, reason, eval, infil..."
39094,"[babygram, chest, port, clip, reason, term, ba..."
56945,"[pm, trauma, port, chest, clip, clip, number, ..."
25670,"[pm, knee, ap, lat, oblique, right, clip, clip..."
6163,"[chest, portable, ap, clip, clip, number, radi..."
...,...
29765,"[video, oropharyngeal, swallow, clip, clip, nu..."
345,"[pm, mr, head, w, w, contrast, mr, angio, head..."
53760,"[chest, portable, ap, clip, clip, number, radi..."
24961,"[knee, ap, lat, oblique, left, clip, clip, num..."


In [3]:
y_train

,G1,G2,G3,G4,G5,G6,G7,G8,G9,G10,G11,G12,G13,G14,G15,G16,G17,G18,G19,G20
10603,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
39094,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1
56945,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
25670,0,1,1,1,0,1,1,1,0,1,0,0,0,0,0,1,0,0,0,0
6163,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29765,0,0,1,0,0,1,1,1,1,0,0,1,1,0,0,0,0,0,0,0
345,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0
53760,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0
24961,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0


In [132]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential,Model
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM,LeakyReLU,Conv1D,Input,Add,Concatenate,concatenate,add
from keras.layers import GlobalMaxPooling1D
from keras.models import Model
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers.merge import Concatenate

import pandas as pd
import numpy as np
import re

import matplotlib.pyplot as plt

In [5]:
WordVectorz=dict(zip(new_model.wv.index2word,new_model.wv.vectors))
class AverageEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim =100 # because we use 100 embedding points 

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])
AverageEmbeddingVectorizer(WordVectorz).fit(x,y)
x.text = AverageEmbeddingVectorizer(WordVectorz).transform(x.text)

/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/pandas/core/generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [6]:
x.shape

(45526, 1)

In [7]:
x

,text
47366,-0.642904
47024,0.425443
9046,-0.259994
843,-0.184132
43466,-0.394579
...,...
24035,-0.042399
30073,-0.329049
25219,0.365235
33394,0.181337


In [9]:
y_train.shape

(38697, 20)

In [128]:
from sklearn.model_selection import train_test_split
seed=243 
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.15,random_state=seed)

In [138]:
model = Sequential()
#input_x = Input(shape=x_train.shape)
model.add(Dense(64))
model.add(LeakyReLU())
model.add(LeakyReLU(alpha=0.1))
model.add(Dense(16))
model.add(LeakyReLU(alpha=0.1))
model.add(Dense(2))
model.add(LeakyReLU())
model.add(Dropout(0.2))
model.add(Dense(16))
model.add(LeakyReLU())
model.add(Dense(64,activation='tanh'))
model.add(Dense(y_train.shape[1],activation='sigmoid'))


In [139]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['acc'])


In [140]:
model.fit(x_train.values,y_train.values,epochs=50,batch_size=128)

ValueError: Error when checking input: expected input_12 to have 3 dimensions, but got array with shape (38697, 1)

In [ ]:
print(model.summary())

In [141]:
y_pred = model.predict(x_test)
y_pred

ValueError: Error when checking input: expected input_12 to have 3 dimensions, but got array with shape (6829, 1)

In [ ]:
y_round = np.round(y_pred)

In [ ]:
y_round

In [ ]:
np.unique(y_round)

In [ ]:
from sklearn.metrics import jaccard_score,roc_auc_score,confusion_matrix,hamming_loss
print(jaccard_score(y_test,y_round,average='micro'))
print(jaccard_score(y_test,y_round,average='macro'))
print(roc_auc_score(y_test,y_round))
print(hamming_loss(y_test,y_round))